## Importo librerías

In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', None)

In [ ]:
!pip install geopandas

In [ ]:
!pip install requests

In [ ]:
# Si se está en google colab, reemplazar por path de Drive
data_path = 'C:/Users/Usuario/Documents/GitHub/enmodo/'

In [ ]:
import sys

# Si se está en google colab, reemplazar por path donde tiene la carpeta "scripts"
sys.path.insert(1, data_path +'scripts')

import eod_analysis as eod

## Google colab

Para ejecutar este notebook en Colab, primero descomenten y ejecuten las siguientes 3 celdas. Luego de ejecutar la notebook se reiniciará.

In [ ]:
'''
# Montando datos desde Google Drive
from google.colab import drive
drive.mount('/content/drive')
'''

In [ ]:
'''
import sys
sys.path.append('/aqui_reemplazar_path_drive/')
'''

### Levanto ENMODO 2009
https://www.argentina.gob.ar/transporte/dgppse/publicaciones/encuestas

Esta notebook apunta directamente a datos subidos a github por lo cual no es necesario reemplazarla por data paths locales o en drive

In [ ]:
hogares = pd.read_csv('https://media.githubusercontent.com/media/RacoFernandez/enmodo/main/buenos-aires/source-csv/hogares.csv')
personas = pd.read_csv('https://media.githubusercontent.com/media/RacoFernandez/enmodo/main/buenos-aires/source-csv/personas.csv')
viajes = pd.read_csv('https://media.githubusercontent.com/media/RacoFernandez/enmodo/main/buenos-aires/source-csv/viajes.csv')
etapas = pd.read_csv('https://media.githubusercontent.com/media/RacoFernandez/enmodo/main/buenos-aires/source-csv/etapas.csv')

### Caracterización de los datos

Los datos utilizados en este cuaderno corresponden a los resultados de la Encuesta Origen-Destino de Buenos Aires (AMBA), Argentina del 2009. 
A partir de estos datos se calcularan indicadores en tres niveles: de Cantidad de Viajes, de Tiempo de Viajes, y de Distancia de Viajes.

Hay datos a nivel hogares, personas, viajes y etapas.

### Importando datos

In [ ]:
import requests
import json
datastring = "https://raw.githubusercontent.com/RacoFernandez/enmodo/main/buenos-aires/source-geojson/departamentos-buenos_aires.json"
resp = requests.get(datastring)
departamentos = json.loads(resp.text)

In [ ]:
datastring = "https://raw.githubusercontent.com/RacoFernandez/enmodo/main/buenos-aires/source-geojson/departamentos-ciudad_autonoma_de_buenos_aires.json"
resp = requests.get(datastring)
departamentos_caba = json.loads(resp.text)

In [ ]:
import geopandas as gpd
departamentos_gpd = gpd.GeoDataFrame.from_features(departamentos["features"])

In [ ]:
departamentos_gpd.departamento.unique()

## Limpieza de la capa de departamentos y join con datos de viajes de ENMODO (ya que la misma no viene con la data de la geometría de los viajes)

In [ ]:
# Me quedo solo con algunos departamentos (los presentes en la ENMODO 2009)
departamentos_amba_gpd = departamentos_gpd[(departamentos_gpd['departamento'].str.contains("BROWN"))|
                  (departamentos_gpd['departamento'].str.contains("MATANZA"))|
                  (departamentos_gpd['departamento'].str.contains("LOPEZ"))|
                  (departamentos_gpd['departamento'].str.contains("ECHEVERRIA"))|
                  (departamentos_gpd['departamento'].str.contains("MORON"))|
                  (departamentos_gpd['departamento'].str.contains("ISIDRO"))|
                  (departamentos_gpd['departamento'].str.contains("PILAR"))|
                  (departamentos_gpd['departamento'].str.contains("LANUS"))|
                  (departamentos_gpd['departamento'].str.contains("JOSE C PAZ"))|
                  (departamentos_gpd['departamento'].str.contains("QUILMES"))|
                  (departamentos_gpd['departamento'].str.contains("HURLINGHAM"))|
                  (departamentos_gpd['departamento'].str.contains("SAN MARTIN"))|
                  (departamentos_gpd['departamento'].str.contains("AVELLANEDA"))|
                  (departamentos_gpd['departamento'].str.contains("LOMAS DE ZAMORA"))|
                  (departamentos_gpd['departamento'].str.contains("BERAZATEGUI"))|
                  (departamentos_gpd['departamento'].str.contains("ITUZAINGO"))|
                  (departamentos_gpd['departamento'].str.contains("ESCOBAR"))|
                  (departamentos_gpd['departamento'].str.contains("SAN MIGUEL"))|
                  (departamentos_gpd['departamento'].str.contains("MERLO"))|
                  (departamentos_gpd['departamento'].str.contains("FEBRERO"))|
                  (departamentos_gpd['departamento'].str.contains("SAN FERNANDO"))|
                  (departamentos_gpd['departamento'].str.contains("MORENO"))|
                  (departamentos_gpd['departamento'].str.contains("EZEIZA"))|
                  (departamentos_gpd['departamento'].str.contains("MALVINAS"))|
                  (departamentos_gpd['departamento'].str.contains("TIGRE"))|
                  (departamentos_gpd['departamento'].str.contains("PERON"))|
                  (departamentos_gpd['departamento'].str.contains("LA PLATA"))|
                  (departamentos_gpd['departamento'].str.contains("CAÑUELAS"))|
                  (departamentos_gpd['departamento'].str.contains("GENERAL LAS HERAS"))|
                  (departamentos_gpd['departamento'].str.contains("ZARATE"))|
                  (departamentos_gpd['departamento'].str.contains("MARCOS PAZ"))|
                  (departamentos_gpd['departamento'].str.contains("BRANDSEN"))|
                  (departamentos_gpd['departamento'].str.contains("ENSENADA"))|
                  (departamentos_gpd['departamento'].str.contains("MERCEDES"))|
                  (departamentos_gpd['departamento'].str.contains("BERISSO"))|
                  (departamentos_gpd['departamento'].str.contains("SAN VICENTE"))|
                  (departamentos_gpd['departamento'].str.contains("GENERAL RODRIGUEZ"))|
                  (departamentos_gpd['departamento'].str.contains("EXALTACION DE LA CRUZ"))|
                  (departamentos_gpd['departamento'].str.contains("FLORENCIO"))]

In [ ]:
del(departamentos_gpd)

In [ ]:
departamentos_caba_gpd = gpd.GeoDataFrame.from_features(departamentos_caba["features"])

In [ ]:
del(departamentos_caba)

In [ ]:
departamentos_amba_gpd = departamentos_amba_gpd.append(departamentos_caba_gpd)

In [ ]:
viajes.partido = viajes.partido.str.upper()

In [ ]:
viajes.TT_PARTIDO_ORIGEN = viajes.TT_PARTIDO_ORIGEN.str.upper()

In [ ]:
viajes.TT_PARTIDO_DESTINO = viajes.TT_PARTIDO_DESTINO.str.upper()

In [ ]:
cols = viajes.select_dtypes(include=[np.object]).columns
viajes[cols] = viajes[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

In [ ]:
dict = {"ALTE. BROWN" : 'ALMIRANTE BROWN', "E. ECHEVERRIA": 'ESTEBAN ECHEVERRIA', "JOSE C. PAZ": 'JOSE C PAZ',
         "PTE. PERON": 'PRESIDENTE PERON', "CORONEL BRANDSEN": 'BRANDSEN', "CANUELAS": 'CAÑUELAS', "SOLDATI": 'VILLA SOLDATI',
        "NUNEZ": 'NUÑEZ'}
viajes=viajes.replace({"partido": dict, "TT_PARTIDO_ORIGEN": dict, "TT_PARTIDO_DESTINO": dict})

In [ ]:
barrios_caba = pd.read_csv('https://cdn.buenosaires.gob.ar/datosabiertos/datasets/barrios/barrios.csv')

In [ ]:
viajes = viajes.merge(barrios_caba, left_on='TT_LOCALIDAD_ORIGEN', right_on='barrio', how='left')

In [ ]:
viajes.pop("WKT")
viajes.pop("barrio")
viajes.pop("perimetro")
viajes.pop("area")

In [ ]:
viajes = viajes.merge(barrios_caba, left_on='TT_LOCALIDAD_DESTINO', right_on='barrio', how='left')

In [ ]:
viajes.pop("WKT")
viajes.pop("barrio")
viajes.pop("perimetro")
viajes.pop("area")

In [ ]:
viajes = viajes.rename(columns={'comuna_x': 'comuna_origen', 'comuna_y': 'comuna_destino'})

In [ ]:
viajes.comuna_origen.fillna(viajes.TT_PARTIDO_ORIGEN, inplace = True)

In [ ]:
viajes.comuna_destino.fillna(viajes.TT_PARTIDO_DESTINO, inplace = True)

In [ ]:
viajes.comuna_origen = viajes.comuna_origen.astype(str)

In [ ]:
viajes.comuna_destino = viajes.comuna_destino.astype(str)

In [ ]:
viajes.comuna_origen.unique()

In [ ]:
dict = {"1.0" : 'COMUNA 1', "2.0" : 'COMUNA 2', "3.0" : 'COMUNA 3', "4.0" : 'COMUNA 4',
        "5.0" : 'COMUNA 5', "6.0" : 'COMUNA 6', "7.0" : 'COMUNA 7', "8.0" : 'COMUNA 8',
        "9.0" : 'COMUNA 9', "10.0" : 'COMUNA 10', "11.0" : 'COMUNA 11', "12.0" : 'COMUNA 12',
        "13.0" : 'COMUNA 13', "14.0" : 'COMUNA 14', "15.0" : 'COMUNA 15'}
viajes=viajes.replace({"comuna_origen": dict, "comuna_destino": dict})

In [ ]:
viajes_gpd = departamentos_amba_gpd.merge(viajes, left_on = 'departamento', right_on='comuna_origen', how='right')

In [ ]:
viajes_gpd = viajes_gpd.merge(departamentos_amba_gpd, left_on = 'comuna_destino', right_on = 'departamento', how='left')

In [ ]:
viajes_gpd = viajes_gpd.rename(columns={'geometry_x': 'geometry_origen', 'geometry_y': 'geometry_destino'})

In [ ]:
viajes_gpd.pop("id_x")
viajes_gpd.pop("departamento_x")
viajes_gpd.pop("cabecera_x")
viajes_gpd.pop("provincia_x")
viajes_gpd.pop("id_y")
viajes_gpd.pop("departamento_y")
viajes_gpd.pop("cabecera_y")
viajes_gpd.pop("provincia_y")

In [ ]:
viajes_gpd = viajes_gpd.set_geometry("geometry_origen")

In [ ]:
viajes_gpd['origen_x_coord'] = viajes_gpd['geometry_origen'].centroid.x
viajes_gpd['origen_y_coord'] = viajes_gpd['geometry_origen'].centroid.y

In [ ]:
viajes_gpd = viajes_gpd.set_geometry("geometry_destino")

In [ ]:
viajes_gpd['destino_x_coord'] = viajes_gpd['geometry_destino'].centroid.x
viajes_gpd['destino_y_coord'] = viajes_gpd['geometry_destino'].centroid.y

In [ ]:
# filtro viajes sin origen o destino especificado
viajes_gpd = viajes_gpd[(viajes_gpd.comuna_origen != 'FUERA DE AREA - NORTE')&(viajes_gpd.comuna_origen != 'FUERA DE AREA - SUR')&(viajes_gpd.comuna_origen != 'SIN DATO')&
            (viajes_gpd.comuna_destino != 'FUERA DE AREA - NORTE')&(viajes_gpd.comuna_destino != 'FUERA DE AREA - SUR')&(viajes_gpd.comuna_destino != 'SIN DATO')]

## Modo

In [ ]:
etapas['Modo'] = etapas['Modo'].replace(['Ferrocarril', 'Subte / Premetro', 'A pie', 'Automovil / conductor', 'Taxi', 'Automovil / acompañante',
       'Colectivo', 'Bicicleta', 'Remis', 'Charter / combi', 'Bus empresa', 'Motocicleta', 'Otros', 'Transporte escolar', 'Lancha Colectiva'],
        ['Ferrocarril', 'Subte/Premetro', 'A pie', 'Automovil/conductor', 'Taxi', 'Automovil/acompañante', 'Colectivo', 'Bicicleta', 'Remis', 'Charter/combi',
       'Bus empresa', 'Motocicleta', 'Otros', 'Transporte escolar', 'Lancha Colectiva'])

In [ ]:
# Creo sub-categorías de agrupación en Etapas

etapas_cats_old = ['Ferrocarril', 'Subte/Premetro', 'A pie', 'Automovil/conductor', 'Taxi', 'Automovil/acompañante', 'Colectivo', 'Bicicleta',
               'Remis', 'Charter/combi', 'Bus empresa', 'Motocicleta', 'Otros', 'Transporte escolar', 'Lancha Colectiva']
etapas_cats_new = ['Público', 'Público', 'No Motorizado', 'Auto/Moto', 'A demanda', 'Auto/Moto', 'Público', 'No Motorizado',
               'A demanda', 'A demanda', 'A demanda', 'Auto/Moto', 'A demanda', 'A demanda', 'A demanda']


etapas['Modo_agg'] = etapas['Modo'].replace(etapas_cats_old, etapas_cats_new)

# Creo variables dummies para agregación de etapas a nivel de viaje
etapas = pd.concat([etapas, pd.get_dummies(etapas.Modo)], axis=1)
etapas = pd.concat([etapas, pd.get_dummies(etapas.Modo_agg)], axis=1)

In [ ]:
particion_etapas = etapas.groupby(['Modo', 'Modo_agg']).wt1.sum().round().astype(int).reset_index()
particion_etapas.columns = ['Modo', 'Modo_agg', 'Etapas']
particion_etapas['Etapas_porc'] = (particion_etapas.Etapas / particion_etapas.Etapas.sum() * 100).round(1)

### Ahora en tabla viajes

In [ ]:
etapas.columns

In [ ]:
viajes_gpd_etapa = viajes_gpd.merge(etapas, on = 'IDV', how='left')

In [ ]:
viajes_gpd_etapa = viajes_gpd_etapa.merge(personas, left_on = 'IDP_x', right_on = 'IDP', how='left')

In [ ]:
viajes_gpd = viajes_gpd.merge(personas, on = 'IDP', how='left')

In [ ]:
'''
# Otra alternativa podría ser:
Viajes_agg = etapas.groupby(['IDH', 'IDP', 'IDV']).agg({'Modo':'max', 'modo_agg':'max', 'Ferrocarril':'sum', 'Subte/Premetro':'sum', 'A pie':'sum', 'Automovil/conductor':'sum', 'Taxi':'sum', 
                                                  'Automovil/acompañante':'sum', 'Colectivo':'sum', 'Bicicleta':'sum', 'Remis':'sum', 'Charter/combi':'sum', 
                                                  'Bus empresa':'sum', 'Motocicleta':'sum', 'Otros':'sum', 'Transporte escolar':'sum', 'Lancha Colectiva':'sum', 
                                                  'Público':'sum', 'No Motorizado':'sum', 'Auto/Moto':'sum', 'A demanda':'sum', 'wt1':'max'}).reset_index()

Viajes_agg['Cant_etapas'] = Viajes_agg[['Público', 'No Motorizado', 'Auto/Moto', 'A demanda']].sum(axis=1)
Viajes_agg.loc[(Viajes_agg.Cant_etapas>1)&(Viajes_agg.Público>0), 'Modo'] = 'Multietapa'
Viajes_agg.loc[(Viajes_agg.Cant_etapas>1)&((Viajes_agg.Modo!='Multietapa')), 'Modo'] = 'Otros Multietapa'
Viajes_agg.loc[Viajes_agg.Modo=='Otros Multietapa', 'Modo_agg'] = 'Otros Multietapa'
Viajes_agg.loc[Viajes_agg.Modo=='Multietapa', 'Modo_agg'] = 'Multietapa'

Viajes_agg['Modo_agg2'] = Viajes_agg.Modo_agg.replace(['No Motorizado', 'Auto/Moto', 'A demanda', 'Público',
       'Otros Multietapa', 'Multietapa'], ['No Motorizado', 'Auto/Moto', 'A demanda', 'Público',
       'A demanda', 'Público'])
'''

## Cálculo distancia

In [ ]:
def manhattan_distance(a, b):
    return sum(abs(e1-e2) for e1, e2 in zip(a,b)) * 100

In [ ]:
def distance_calc (row):
    start = (row['origen_y_coord'], row['origen_x_coord'])
    stop = (row['destino_y_coord'], row['destino_x_coord'])

    return manhattan_distance(start, stop)

In [ ]:
viajes_gpd['distance'] = viajes_gpd.apply (lambda row: distance_calc (row),axis=1)

In [ ]:
viajes_gpd_etapa['distance'] = viajes_gpd_etapa.apply (lambda row: distance_calc (row),axis=1)

## Cálculo de tiempo

In [ ]:
def time_difference(row):
  if row['HORALLEG'] == 99999.0:
    return(0)
  elif((row['HORALLEG'] >= 0) and (row['HORASALI'] <= 23 and row['HORASALI'] >= 20)):
    time_diff = (24 - row['HORASALI']) * 60 + (row['MINLLEGA'] - row['MINSALID'])
  elif(row['HORALLEG'] == 1 and (row['HORASALI'] <= 23 and row['HORASALI'] >= 20)):
    time_diff = (25 - row['HORASALI']) * 60 + (row['MINLLEGA'] - row['MINSALID'])
  else:
    time_diff = (row['HORALLEG'] - row['HORASALI']) * 60 + (row['MINLLEGA'] - row['MINSALID'])
  return(time_diff)

In [ ]:
viajes_gpd['time_diff'] = viajes_gpd.apply (lambda row: time_difference (row),axis=1)

In [ ]:
viajes_gpd_etapa['time_diff'] = viajes_gpd_etapa.apply (lambda row: time_difference (row),axis=1)

## Horario punta

In [ ]:
def tipo_horario(row):
    if (row['HORASALI'] >= 7.0 and row['HORASALI'] < 11) or (row['HORASALI'] >= 16 and row['HORASALI'] < 20) :
        return 'punta'
    else:
        return 'valle'

In [ ]:
viajes_gpd['tipo_horario'] = viajes_gpd.apply(tipo_horario, axis=1)

In [ ]:
viajes_gpd_etapa['tipo_horario'] = viajes_gpd_etapa.apply(tipo_horario, axis=1)

## Horario punta mañana

In [ ]:
def tipo_horario_punta_manana(row):
    if (row['HORASALI'] >= 7.0 and row['HORASALI'] < 11) :
        return True
    else:
        return False

In [ ]:
viajes_gpd['punta_manana'] = viajes_gpd.apply(tipo_horario_punta_manana, axis=1)

In [ ]:
viajes_gpd_etapa['punta_manana'] = viajes_gpd_etapa.apply(tipo_horario_punta_manana, axis=1)

## Recategorizo modo para el posterior calculo de indicadores

In [ ]:
viajes_gpd_etapa['Modo_indicador'] = viajes_gpd_etapa['Modo']

In [ ]:
dict = {"Automovil/acompañante" : 'Auto', "Automovil/conductor": 'Auto', "Bus empresa": 'Otros',
         "Charter/combi": 'Otros', "Colectivo": 'Público', "Ferrocarril": 'Público', "Lancha Colectiva": 'Público',
        "Remis": 'Otros', "Subte/Premetro": 'Público', "Taxi": 'Otros', "Transporte escolar": 'Otros'}

viajes_gpd_etapa=viajes_gpd_etapa.replace({"Modo_indicador": dict})

In [ ]:
# Cambio valores lugar de trabajo
viajes_gpd['ACTIORIG_AGG'] = viajes_gpd['ACTIORIG'] 
dict = {'Asunto laboral': 'Trabajo', 'Lugar de trabajo': 'Trabajo'}
viajes_gpd=viajes_gpd.replace({"ACTIORIG_AGG": dict})

## Join de Viajes - Etapa + Hogares

In [ ]:
viajes_gpd_etapa_hogares = viajes_gpd_etapa.merge(hogares, on = 'IDH', how='left')

In [ ]:
viajes_gpd_hogares = viajes_gpd.merge(hogares, left_on = 'IDH_x', right_on = 'IDH', how='left')

## Generación de csv

In [ ]:
viajes_gpd_hogares.shape

In [ ]:
viajes_gpd_etapa_hogares.shape

In [ ]:
viajes_gpd_hogares_to_save = viajes_gpd_hogares.drop(['geometry_origen', 'geometry_destino'], axis=1)


In [ ]:
viajes_gpd_hogares_to_save = pd.DataFrame(viajes_gpd_hogares_to_save)

In [ ]:
viajes_gpd_hogares_to_save.to_csv(data_path + 'buenos-aires/output-csv/viajes_gpd_hogares.csv', index=False)

In [ ]:
viajes_gpd_etapa_hogares_to_save = viajes_gpd_etapa_hogares.drop(['geometry_origen', 'geometry_destino'], axis=1)

In [ ]:
viajes_gpd_etapa_hogares_to_save = pd.DataFrame(viajes_gpd_etapa_hogares_to_save)

In [ ]:
viajes_gpd_etapa_hogares_to_save = viajes_gpd_etapa_hogares_to_save.loc[:,~viajes_gpd_etapa_hogares_to_save.columns.duplicated()]

In [ ]:
viajes_gpd_etapa_hogares_to_save = viajes_gpd_etapa_hogares_to_save.drop(['Unnamed: 0_x', 'HORAINI', 'HORAFIN', 'wt1_x', 'z_frecuenc_imput_x', 'Unnamed: 0_y', 'wt1_y', 'IDH_y',
                                       'IDP_y', 'Unnamed: 0_x', 'wt1_x', 'CDNINGUN', 'CDMOTORA', 'CDMENTAL', 'CDVISUAL', 'CDAUDITI', 'CDHABLA', 'CDOTRAS', 'z_cocherasino',
                                       'z_tieneciclomo', 'z_tienemoto', 'z_tieneauto', 'z_gastoco2', 'z_gastoma2', 'A pie', 'Automovil/acompañante', 'Automovil/conductor',
                                       'Bicicleta', 'Bus empresa', 'Charter/combi', 'Colectivo', 'Ferrocarril', 'Lancha Colectiva', 'Motocicleta', 'Otros', 'Remis', 'Subte/Premetro', 'Taxi', 'Transporte escolar', 'A demanda', 
                                       'Auto/Moto', 'No Motorizado', 'TIPOGARA', 'TIPOCOST', 'COSTOPEA', 'COSTOTAX', 'CUACAANT', 'CUACADES', 'ESPEHORA', 'ESPEMINU', 'DURAHORA', 'DURAMINU',
                                       'IDH_x', 'IDP_x', 'origen_x_coord', 'origen_y_coord', 'destino_x_coord', 'destino_y_coord'], axis=1)

In [ ]:
viajes_gpd_hogares_to_save.shape

In [ ]:
viajes_gpd_etapa_hogares_to_save.shape

In [ ]:
viajes_gpd_etapa_hogares_to_save.to_csv(data_path + 'output-csv/viajes_gpd_etapa_hogares.csv', index=False)

## Generación de geojson

### viajes

In [ ]:
origenes_viajes = pd.DataFrame(viajes_gpd_hogares)
destinos_viajes = pd.DataFrame(viajes_gpd_hogares)

In [ ]:
origenes_viajes = gpd.GeoDataFrame(origenes_viajes, geometry='geometry_origen')
destinos_viajes = gpd.GeoDataFrame(destinos_viajes, geometry='geometry_destino')

In [ ]:
origenes_viajes.geometry_origen = origenes_viajes.geometry_origen.centroid
destinos_viajes.geometry_destino = destinos_viajes.geometry_destino.centroid

In [ ]:
origenes_viajes = origenes_viajes.loc[:,~origenes_viajes.columns.duplicated()]
destinos_viajes = destinos_viajes.loc[:,~destinos_viajes.columns.duplicated()]

In [ ]:
origenes_viajes.drop(['geometry_destino'], axis=1, inplace = True)
destinos_viajes.drop(['geometry_origen'], axis=1, inplace = True)

In [ ]:
origenes_viajes.to_file(data_path + 'output-geojson/origenes_viajes.geojson', driver='GeoJSON')
destinos_viajes.to_file(data_path + 'output-geojson/destinos_viajes.geojson', driver='GeoJSON')

### viajes_etapa

In [ ]:
origenes_viajes_etapa = pd.DataFrame(viajes_gpd_etapa_hogares)
destinos_viajes_etapa = pd.DataFrame(viajes_gpd_etapa_hogares)

In [ ]:
origenes_viajes_etapa = gpd.GeoDataFrame(origenes_viajes_etapa, geometry='geometry_origen')
destinos_viajes_etapa = gpd.GeoDataFrame(destinos_viajes_etapa, geometry='geometry_destino')

In [ ]:
origenes_viajes_etapa.geometry_origen = origenes_viajes_etapa.geometry_origen.centroid
destinos_viajes_etapa.geometry_destino = destinos_viajes_etapa.geometry_destino.centroid

In [ ]:
origenes_viajes_etapa = origenes_viajes_etapa.loc[:,~origenes_viajes_etapa.columns.duplicated()]
destinos_viajes_etapa = destinos_viajes_etapa.loc[:,~destinos_viajes_etapa.columns.duplicated()]

In [ ]:
origenes_viajes_etapa.drop(['geometry_destino'], axis=1, inplace = True)
destinos_viajes_etapa.drop(['geometry_origen'], axis=1, inplace = True)

In [ ]:
origenes_viajes_etapa.to_file(data_path + 'output-geojson/origenes_viajes_etapa.geojson', driver='GeoJSON')
destinos_viajes_etapa.to_file(data_path + 'output-geojson/destinos_viajes_etapa.geojson', driver='GeoJSON')

## Generación de shapefile

### viajes

In [ ]:
origenes_viajes.to_file(data_path + 'output-shp/origenes_viajes.shp', driver='GeoJSON')
destinos_viajes.to_file(data_path + 'output-shp/destinos_viajes.shp', driver='GeoJSON')

### viajes_etapa

In [ ]:
origenes_viajes_etapa.to_file(data_path + 'output-shp/origenes_viajes_etapa.shp', driver='GeoJSON')
destinos_viajes_etapa.to_file(data_path + 'output-shp/destinos_viajes_etapa.shp', driver='GeoJSON')

## Cálculo de indicadores

#### Distancia de viajes en automóvil privado + Distancia de viajes en transporte público

In [ ]:
viajes_gpd_etapa.groupby(['Modo_agg']).mean('distance')['distance']

In [ ]:
viajes_gpd_etapa.groupby(['Modo_agg', 'SEXO']).mean('distance')['distance']

#### Distancia de viajes por motivo estudio + Distancia de viajes por motivo trabajo

In [ ]:
viajes_gpd.groupby(['ACTIORIG_AGG']).mean('distance')['distance']

In [ ]:
 viajes_gpd.groupby(['ACTIORIG_AGG', 'SEXO']).mean('distance')['distance']

#### Proporción de viajes intrazonales versus interzonales

In [ ]:
viajes_gpd["es_caba_origen"] = viajes_gpd["comuna_origen"].str.contains("COMUNA")

In [ ]:
viajes_gpd["es_caba_destino"] = viajes_gpd["comuna_destino"].str.contains("COMUNA")

In [ ]:
viajes_gpd.loc[((viajes_gpd['es_caba_origen'] == True) & (viajes_gpd['es_caba_destino'] == True)) |
               ((viajes_gpd['es_caba_origen'] == False) & (viajes_gpd['es_caba_destino'] == False)), 'tipo_viaje_zona'] = 'Intrazonal'  

In [ ]:
viajes_gpd.loc[((viajes_gpd['es_caba_origen'] == True) & (viajes_gpd['es_caba_destino'] == False)) |
               ((viajes_gpd['es_caba_origen'] == False) & (viajes_gpd['es_caba_destino'] == True)), 'tipo_viaje_zona'] = 'Interzonal'  

In [ ]:
viajes_gpd.groupby('tipo_viaje_zona').size()

In [ ]:
viajes_gpd.shape

#### Tiempo promedio de viaje al trabajo en transporte público


In [ ]:
viajes_gpd_etapa.groupby(['Modo_agg']).mean('time_diff')['time_diff']

In [ ]:
viajes_gpd_etapa.groupby(['Modo_agg', 'SEXO']).mean('time_diff')['time_diff']

#### Tiempo promedio de viaje en hora punta de mañana

In [ ]:
viajes_gpd.groupby(['tipo_horario']).mean('time_diff')['time_diff']

In [ ]:
viajes_gpd.groupby(['tipo_horario', 'SEXO']).mean('time_diff')['time_diff']

#### Tiempo de viaje en transporte público en hora punta en la mañana


In [ ]:
viajes_gpd_etapa.groupby(['Modo_agg', 'punta_manana']).mean('time_diff')['time_diff']

In [ ]:
viajes_gpd_etapa.groupby(['Modo_agg', 'punta_manana', 'SEXO']).mean('time_diff')['time_diff']

#### Brecha de tiempo entre hora punta y valle del transporte público

In [ ]:
viajes_gpd_etapa.groupby(['Modo_agg', 'tipo_horario']).mean('time_diff')['time_diff']

In [ ]:
viajes_gpd_etapa.groupby(['Modo_agg', 'tipo_horario', 'SEXO']).mean('time_diff')['time_diff']

#### Tiempo promedio de viaje por modo y tipo de transporte

In [ ]:
viajes_gpd_etapa.groupby(['Modo_agg']).mean('time_diff')['time_diff']

In [ ]:
viajes_gpd_etapa.groupby(['Modo_agg', 'SEXO']).mean('time_diff')['time_diff']

In [ ]:
viajes_gpd_etapa.groupby(['Modo']).mean('time_diff')['time_diff']

In [ ]:
viajes_gpd_etapa.groupby(['Modo_indicador']).mean('time_diff')['time_diff']

In [ ]:
viajes_gpd_etapa.time_diff.mean()

#### Distribución y Participación Modal de los viajes por clasificador socioeconómico


In [ ]:
viajes_gpd_etapa_hogares.groupby('Modo')['z_Quintil_ing_per_capita_AD_EQUIV'].value_counts(normalize=True)

In [ ]:
viajes_gpd_etapa_hogares.groupby('Modo')['z_Quintil_ing_per_capita_AD_EQUIV'].value_counts(normalize=False)

Proporción entre los viajes en transporte público y privado por clasificador socioeconómico


In [ ]:
viajes_gpd_etapa_hogares.groupby('Modo_indicador')['z_Quintil_ing_per_capita_AD_EQUIV'].value_counts(normalize=True)

In [ ]:
viajes_gpd_etapa.groupby('Modo_indicador')['ORDEN_y'].value_counts(normalize=True)

Tasa promedio de viajes diarios en transporte privado por clasificador económico

Cantidad de pasajeros por modo (demanda) por cada cien mil habitantes

Porcentaje de la población dentro del área de influencia de la red de transporte público


Tasa de buses de transporte público por cada cien mil habitantes

Cantidad de vehículos de transporte privado

Cantidad de vehículos del transporte público impulsados por energías no fósiles (eléctricos)

Tasa de muertes en accidentes de tráfico

Peso monetario del transporte público en el presupuesto familiar

Brecha entre hombres y mujeres de viajes en la noche respecto al total de viajes


Relación combustible y costo ticket transporte público


## Coeficiente de movilidad (accesibilidad e interacción) y de inmovilidad

In [ ]:
!pip3 install seaborn

In [ ]:
import seaborn as sns
from scipy import stats
from scipy import sparse
import scipy.sparse.linalg.dsolve as linsolve
from sympy import *

In [ ]:
cantidad_viajes_etapa_por_sexo = viajes_gpd_etapa.groupby(['IDP_x', 'SEXO']).size()

In [ ]:
tiempo_prom_viajes_etapa_por_sexo = viajes_gpd_etapa.groupby(['IDP_x', 'SEXO']).mean('time_diff')['time_diff']

In [ ]:
cantidad_viajes_etapa_por_sexo = pd.DataFrame(cantidad_viajes_etapa_por_sexo)

In [ ]:
tiempo_prom_viajes_etapa_por_sexo = pd.DataFrame(tiempo_prom_viajes_etapa_por_sexo)

In [ ]:
n_viajes = pd.merge(cantidad_viajes_etapa_por_sexo, tiempo_prom_viajes_etapa_por_sexo, left_index=True, right_index=True)

In [ ]:
n_viajes.columns = ['n_viajes', 'tiempo_total']

In [ ]:
n_viajes['PONDERADOR_CALIBRADO'] = 1

In [ ]:
n_viajes.reset_index(inplace = True)

In [ ]:
x, y = symbols('x y')
eod.plot_lmplot(n_viajes)

In [ ]:
groups = eod.generate_groups(n_viajes)

In [ ]:
eod.plot_lmplot(n_viajes, col="SEXO", hue="SEXO", col_wrap=1)

In [ ]:
subgroups = n_viajes.SEXO.unique()
for element in subgroups:
    print(element)
    groups = eod.generate_groups(n_viajes[n_viajes.SEXO==element])
    print(eod.calculate_indicators(groups,'PONDERADOR_CALIBRADO'))